In [54]:
import  os
import numpy as np
import pandas as pd
import scipy.stats
import sklearn


In [282]:
class DataModels:
    data=[]
    results=[]
    x_train=[]
    x_test=[]
    y_train=[]
    y_test=[]
    filepath='Regression_data'
    
    def __init__(self):
        filepath='Regression_data'
        load_data=np.genfromtxt(os.path.join(filepath, filename), delimiter=',' ,\
                             usecols=np.arange(unpredictable, columns), skip_header=skiprows)
        
        self.data=load_data[:,:-1]
        self.results = load_data[:,-1]
        
    def pre_process(self):
        
        #self.data=self.missing_values(self.data)
        self.x_train, self.x_test,\
        self.y_train, self.y_test =sklearn.model_selection.train_test_split(self.data, self.results, test_size=0.3, \
                                                                                random_state=15)
        scaler=sklearn.preprocessing.StandardScaler().fit(self.x_train)
        self.x_train=scaler.transform(self.x_train)
        self.x_test=scaler.transform(self.x_test)
        
    def missing_values(self, data):
        imputer = sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='mean')
        imputer.fit(data)
        return imputer.transform(data)
        
class Communities(DataModels):
    
    def __init__(self):
        
        load_data=np.genfromtxt(os.path.join(self.filepath, 'communities.data'), delimiter=',' ,\
                                usecols=np.arange(5, 128), skip_header=0)
        
        self.data=load_data[:,:-1]
        self.data=self.missing_values(self.data)
        self.results = load_data[:,-1]
        self.pre_process()
        
class Bike(DataModels):
    def __init__(self):
        
        load_data=np.loadtxt(os.path.join(self.filepath, 'hour.csv'), delimiter=',' ,\
                             usecols=np.arange(2, 17), skiprows=1)
        
        self.data=load_data[:,:-1]
        self.results = load_data[:,-1]
        self.pre_process()

class Facebook(DataModels):
    def __init__(self):
        
        def to_int(s):
            if s.decode('utf-8')=="Photo":
                return 0
            elif s.decode('utf-8')=="Status":
                return 1
            elif s.decode('utf-8')=="Link":
                return 2
            else:
                return 3
        
        load_data=np.genfromtxt(os.path.join(self.filepath, 'dataset_Facebook.csv'), delimiter=';' , converters={1:to_int},\
                                skip_header=1, dtype=np.int32 )
       
        self.data=load_data[:, :-1]
        self.results = load_data[:,-1]
        self.pre_process()
        
class Concrete(DataModels):
    def __init__(self):
        
        load_data=np.loadtxt(os.path.join(self.filepath, 'Concrete_data.csv'), delimiter=',' ,\
                                 skiprows=1)
        
        self.data=load_data[:,:-1]
        self.results = load_data[:,-1]
        self.pre_process()
        
class Student(DataModels):
    def __init__(self):
        
        load_data=np.loadtxt(os.path.join(self.filepath, 'student-por.csv'), delimiter=';' ,\
                                 usecols=(2, 6, 7,12, 13, 14, 23,24,25,26,27,28,29,30,31,32 ),\
                              skiprows=1, dtype=np.int32, converters={30: lambda x: int(x.decode('utf-8').strip("\"")),\
                                                                      31:lambda x: int(x.decode('utf-8').strip("\""))})
        
        self.data=load_data[: ,:-1]
        self.results = load_data[:,-1]
        self.pre_process()
        
class Sgemm(DataModels):
    def __init__(self):
        
        load_data=np.genfromtxt(os.path.join(self.filepath, 'sgemm_product.csv'), delimiter=',' ,\
                                 skip_header=1)
        
        self.data=load_data[:,:14]
        self.results = load_data[:,14:-1]
        self.pre_process()
        
class WineQuality(DataModels):
    def __init__(self):
        
        load_data1=np.loadtxt(os.path.join(self.filepath, 'winequality-red.csv'), delimiter=';' ,\
                                 skiprows=1)
        load_data2=np.loadtxt(os.path.join(self.filepath, 'winequality-white.csv'), delimiter=';' ,\
                                 skiprows=1)
        load_data=np.vstack((load_data1, load_data2))
        self.data=load_data[:,:-1]
        self.results = load_data[:,-1]
        self.pre_process()
        

        
class Qsar(DataModels):
    def __init__(self):
        
        load_data=np.loadtxt(os.path.join(self.filepath, 'qsar_aquatic_toxicity.csv'), delimiter=';' ,\
                                skiprows=0)
        
        self.data=load_data[:,:-1]
        self.results = load_data[:,-1]
        self.pre_process()

In [283]:
obj=[]
obj.append(Communities())
obj.append(Bike())
obj.append(Facebook())
obj.append(WineQuality())
obj.append(Sgemm())
obj.append(Qsar())
obj.append(Concrete())
obj.append(Student())


array([[18,  4,  4, ...,  4,  0, 11],
       [17,  1,  1, ...,  2,  9, 11],
       [15,  1,  1, ...,  6, 12, 13],
       ...,
       [18,  1,  1, ...,  6, 11, 12],
       [17,  3,  1, ...,  6, 10, 10],
       [18,  3,  2, ...,  4, 10, 11]])